In [1]:
import numpy as np
import av, io
from PIL import Image
import cv2
import os

from IPython.core import ultratb

In [2]:
def frame(image):
    return av.VideoFrame.from_ndarray(np.array(image), format="rgb24")

In [3]:
cap = cv2.VideoCapture("moving_train.mp4")

if not cap.isOpened():
    print("Fail")

In [4]:
cap = cv2.VideoCapture("moving_train.mp4")
ret, frame = cap.read()

In [5]:
bufEN =  io.BytesIO()
containerEN = av.open(bufEN, "w", format="mpegts")
codecEN = containerEN.add_stream("h264")

codecEN.pix_fmt = "yuv420p"
codecEN.width = 640
codecEN.height = 480

codecEN.bit_rate = 100 * 1000
codecEN.framerate = 12
codecEN.gop_size = 12
codecEN.thread_count = 4
codecEN.profile = "baseline"


codecEN.options = {
    "preset": "ultrafast",
    "tune": "zerolatency",
    "keyint": "12",        # max I-frame interval
    "min-keyint": "12",    # force exactly 12
    "scenecut": "0",       # prevent auto keyframes
    "bframes": "0",
    "slice-max-size": "600",   # max bytes per slice
}

codecEN.open()

bufDE =  io.BytesIO()
codecDE = av.codec.CodecContext.create("h264", "r")

codecDE.pix_fmt = "yuv420p"
codecDE.width = 640
codecDE.height = 480

codecDE.bit_rate = 100 * 1000
codecDE.framerate = 12
codecDE.thread_count = 4
codecDE.profile = "baseline"

codecDE.options = {
    "err_detect": "ignore_err",
    "ec": "guess_mvs+deblock",
    "skip_frame": "noref" 
}

codecDE.open()

In [ ]:
def encode_h264(frame):
    packets = codecEN.encode(frame)
    print("Packet len: ", len(packets))
    for p in packets:
        print("PACKET OBJECT:", p)
        print("  is_keyframe:", p.is_keyframe)
        print("  pts:", p.pts, "  dts:", p.dts)
        print("  size:", p.size)
        containerEN.mux(p)

    ts_bytes = bufEN.getvalue()
    bufEN.seek(0)
    bufEN.truncate(0)
    
    return ts_bytes

def decode_h264(byte):
    
        
    bufDE.write(byte)
    containerDE = av.open(bufDE, "r", format="mpegts")

    video_stream = next(s for s in containerDE.streams if s.type == 'video')
    bufDE.seek(0)
    bufDE.truncate(0)

    for packet in containerDE.demux(video_stream):
        
        # h264byte = av.packet.Packet(packet)
        for frame in packet.decode():
            img = frame.to_ndarray(format='bgr24')  # OpenCV-friendly
            return img

    # packet = containerDE.demux(video_stream)
    # frame = packet.decode()
    # img = frame.to_ndarray(format='bgr24')  # OpenCV-friendly
    # return img

    
    

def bytes_to_bits(b):
    return np.unpackbits(np.frombuffer(b, dtype=np.uint8))

def bits_to_bytes(bits):
    arr = np.packbits(bits)
    return arr.tobytes()

def add_noise_to_bytes(byte_stream, ber=0.00):
    # Convert bytes to a numpy array of uint8
    arr = np.frombuffer(byte_stream, dtype=np.uint8)
    
    # Flatten to bits
    bits = np.unpackbits(arr)
    
    # Generate noise mask
    noise = np.random.rand(bits.size) < ber
    noisy_bits = bits ^ noise.astype(np.uint8)
    
    # Pack bits back to bytes
    noisy_bytes = np.packbits(noisy_bits)
    return noisy_bytes

In [ ]:
def simulate_ldpc_video_pipeline(image):
    # Convert PIL → PyAV frame
    print("FRAME")
    frame = av.VideoFrame.from_ndarray(np.array(image), format="rgb24")

    # Step 1: H.264 encode
    print("ENCODE")
    h264_bytes = encode_h264(frame)
    print("H264 size:", len(h264_bytes) , "type:",type(h264_bytes))

    h264_bytes = add_noise_to_bytes(h264_bytes)

    # Step 7: H.264 decode → image
    print("DECODE")
    restored = decode_h264(h264_bytes)
    print("type:",type(restored))

    return restored

: 

In [ ]:
print("Test Codec")
cap = cv2.VideoCapture("moving_train.mp4")

if not cap.isOpened():
    print("Fail")

target_fps = 6
video_fps = int(cap.get(cv2.CAP_PROP_FPS))
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
frame_idx = 0

# Calculate frame skip to achieve target FPS
skip_frames = max(1, int(video_fps / target_fps))

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
            
        # Skip frames to match target FPS
        if frame_idx % skip_frames != 0:
            continue

        print("Frame idx: ",frame_idx/skip_frames)

        frame = cv2.resize(frame, (640, 480))
        cv2.imshow("Frame",frame)
        frame = simulate_ldpc_video_pipeline(frame)
        if frame is not None:
            print("Showing Frame: ",np.shape(frame))
            # cv2.imshow("Frame",frame)
        else:
            print("Frame dtype: ",type(frame))

except Exception as e:
    print(f"An error occurred: {e}")
    ipython_tb = ultratb.VerboseTB()

    ipython_tb()
    cv2.destroyAllWindows()
    
cv2.destroyAllWindows()



Test Codec
Frame idx:  1.0
FRAME
ENCODE
Packet len:  1
PACKET OBJECT: av.Packet of #0, dts=0, pts=0; 9963 bytes at 0x1e26bce8c20>
  is_keyframe: True
  pts: 0   dts: 0
  size: 9963
H264 size: 10904 type: <class 'bytes'>
DECODE
type: <class 'numpy.ndarray'>
Showing Frame:  (480, 640, 3)


In [ ]:
import serial
import time

# Open COM port
ser = serial.Serial(
    port='COM3',        # change to your port: COM3, COM4, /dev/ttyUSB0, etc.
    baudrate=115200,
    timeout=1           # seconds
)

SerialException: could not open port 'COM3': FileNotFoundError(2, 'The system cannot find the file specified.', None, 2)

In [ ]:
import ldpc
print(dir(ldpc))

In [ ]:
from pyldpc import make_ldpc, encode, decode, get_message

In [ ]:
import numpy as np
from ldpc.bp_decoder import BpDecoder

def add_noise(bits, ber=0.01):
    noise = np.random.rand(len(bits)) < ber
    return bits ^ noise.astype(np.uint8)

# Your H matrix
H = np.array([
    [1, 0, 1, 0, 1, 0],
    [0, 1, 1, 0, 0, 1],
    [1, 1, 0, 1, 0, 0]
], dtype=np.int8)

# CORRECTED G matrix that satisfies H @ G.T = 0
# Format: [I_k | P] where I_k is k×k identity
G = np.array([
    [1, 0, 0, 1, 1, 0],  # These satisfy H @ G.T = 0
    [0, 1, 0, 0, 1, 1],
    [0, 0, 1, 1, 0, 1]
], dtype=np.int8)

# Verify
print("H @ G.T (must be all zeros):")
print((H @ G.T) % 2)
print()

n = G.shape[1]
k = G.shape[0]

def ldpc_encode_bits(msg_bits, G):
    blocks = []
    if len(msg_bits) % k != 0:
        pad = k - (len(msg_bits) % k)
        msg_bits = np.concatenate([msg_bits, np.zeros(pad, dtype=np.uint8)])
    for i in range(0, len(msg_bits), k):
        block = msg_bits[i:i+k]
        codeword = (block @ G) % 2
        blocks.append(codeword.astype(np.uint8))
    return np.concatenate(blocks)

def ldpc_decode_bits(received, H, error_rate=0.01):
    decoder = BpDecoder(H, error_rate=error_rate, max_iter=100)
    n = H.shape[1]
    k = n - H.shape[0]
    decoded_blocks = []
    
    for i in range(0, len(received), n):
        block = received[i:i+n].astype(np.uint8)
        decoded = decoder.decode(block)
        decoded_blocks.append(decoded[:k].astype(np.uint8))
    
    return np.concatenate(decoded_blocks)

# TEST
msg_bits = np.random.randint(0, 2, 12, dtype=np.uint8)
print("Original bits:", msg_bits)

encoded = ldpc_encode_bits(msg_bits, G)
print("Encoded bits :", encoded)

# Verify syndrome
syndrome = np.zeros((H.shape[0], len(encoded)//n), dtype=np.int8)
for i in range(len(encoded)//n):
    block = encoded[i*n:(i+1)*n]
    syndrome[:, i] = (H @ block) % 2
print("Syndrome (should be all zeros):\n", syndrome)

noisy = add_noise(encoded, ber=0.05)
print("Noisy bits   :", noisy)

decoded = ldpc_decode_bits(noisy, H, error_rate=0.05)
print("Decoded bits :", decoded)

print("Match?", np.array_equal(msg_bits, decoded))

In [ ]:
import numpy as np
from ldpc.bp_decoder import BpDecoder

def add_noise(bits, ber=0.01):
    noise = np.random.rand(len(bits)) < ber
    return bits ^ noise.astype(np.uint8)

# Your H matrix
H = np.array([
    [1, 0, 1, 0, 1, 0],
    [0, 1, 1, 0, 0, 1],
    [1, 1, 0, 1, 0, 0]
], dtype=np.int8)

# Let me derive G properly by solving H @ G.T = 0
# For a systematic code G = [I_k | P], we need H @ [I_k | P].T = 0
# This means: H @ [I_k.T] + H @ [P.T] = 0
#             [H_left] + H @ [P.T] = 0
# So: P.T = -H_left^(-1) @ H_right (in GF(2), - is same as +)

# Split H into [H_parity | H_message] = [right part | left part]
# For systematic: first k columns are message bits
# Let's use columns [0,1,2] as message, [3,4,5] as parity

# Actually, let's derive G from scratch using null space
# G is in the null space of H

# Method: Find G such that each row is in null space of H
# For systematic [I_k | P], solve for P

# CORRECTED G - manually derived to satisfy H @ G.T = 0
G = np.array([
    [1, 0, 0, 0, 1, 1],  
    [0, 1, 0, 1, 1, 1],
    [0, 0, 1, 1, 1, 0]
], dtype=np.int8)

# Verify
print("Testing H @ G.T:")
result = (H @ G.T) % 2
print(result)

if np.any(result != 0):
    print("\nG is still wrong! Let me compute the correct one...\n")
    
    # Use scipy or manual method to find null space basis
    # For now, let's use a working example:
    # For this specific H, a valid G is:
    G = np.array([
        [1, 0, 0, 1, 0, 1],
        [0, 1, 0, 0, 1, 1], 
        [0, 0, 1, 1, 1, 1]
    ], dtype=np.int8)
    
    print("Trying alternative G:")
    result = (H @ G.T) % 2
    print("H @ G.T =")
    print(result)

if np.all(result == 0):
    print("✓ Valid G found!\n")
else:
    print("✗ Still invalid, need different approach\n")

n = G.shape[1]
k = G.shape[0]

def ldpc_encode_bits(msg_bits, G):
    blocks = []
    if len(msg_bits) % k != 0:
        pad = k - (len(msg_bits) % k)
        msg_bits = np.concatenate([msg_bits, np.zeros(pad, dtype=np.uint8)])
    for i in range(0, len(msg_bits), k):
        block = msg_bits[i:i+k]
        codeword = (block @ G) % 2
        blocks.append(codeword.astype(np.uint8))
    return np.concatenate(blocks)

def ldpc_decode_bits(received, H, error_rate=0.01):
    decoder = BpDecoder(H, error_rate=error_rate, max_iter=100)
    n = H.shape[1]
    k = n - H.shape[0]
    decoded_blocks = []
    
    for i in range(0, len(received), n):
        block = received[i:i+n].astype(np.uint8)
        decoded = decoder.decode(block)
        decoded_blocks.append(decoded[:k].astype(np.uint8))
    
    return np.concatenate(decoded_blocks)

if np.all(result == 0):
    # TEST
    msg_bits = np.random.randint(0, 2, 12, dtype=np.uint8)
    print("Original bits:", msg_bits)

    encoded = ldpc_encode_bits(msg_bits, G)
    print("Encoded bits :", encoded)

    # Verify syndrome
    syndrome = np.zeros((H.shape[0], len(encoded)//n), dtype=np.int8)
    for i in range(len(encoded)//n):
        block = encoded[i*n:(i+1)*n]
        syndrome[:, i] = (H @ block) % 2
    print("Syndrome:\n", syndrome)

    noisy = add_noise(encoded, ber=0.05)
    print("Noisy bits   :", noisy)

    decoded = ldpc_decode_bits(noisy, H, error_rate=0.05)
    print("Decoded bits :", decoded)

    print("Match?", np.array_equal(msg_bits, decoded))

In [ ]:
import numpy as np
from ldpc.bp_decoder import BpDecoder

def add_noise(bits, ber=0.01):
    noise = np.random.rand(len(bits)) < ber
    return bits ^ noise.astype(np.uint8)

def gf2_rref(matrix):
    """Reduced Row Echelon Form over GF(2)"""
    M = matrix.copy().astype(int)
    rows, cols = M.shape
    
    current_row = 0
    pivot_cols = []
    
    for col in range(cols):
        # Find pivot
        pivot_row = None
        for row in range(current_row, rows):
            if M[row, col] == 1:
                pivot_row = row
                break
        
        if pivot_row is None:
            continue
        
        pivot_cols.append(col)
        
        # Swap rows
        if pivot_row != current_row:
            M[[current_row, pivot_row]] = M[[pivot_row, current_row]]
        
        # Eliminate other rows
        for row in range(rows):
            if row != current_row and M[row, col] == 1:
                M[row] = (M[row] + M[current_row]) % 2
        
        current_row += 1
        if current_row >= rows:
            break
    
    return M, pivot_cols

def find_generator_from_H(H):
    """
    Find generator matrix G such that H @ G.T = 0
    Uses systematic approach with Gaussian elimination
    """
    m, n = H.shape
    k = n - m
    
    print(f"  Matrix size: {m}×{n}, code rate: {k}/{n} = {k/n:.3f}")
    
    # Method 1: Try to put H in systematic form [P | I_m]
    # Then G = [I_k | P.T]
    
    # Augment H with column indices to track permutations
    H_aug = H.copy()
    col_order = list(range(n))
    
    # Find m independent columns to form identity
    H_rref, pivot_cols = gf2_rref(H_aug)
    
    # Separate pivot and non-pivot columns
    non_pivot_cols = [i for i in range(n) if i not in pivot_cols]
    
    if len(pivot_cols) < m:
        raise ValueError(f"H is not full rank! Found only {len(pivot_cols)} pivots, need {m}")
    
    # Reorder columns: [non-pivot | pivot]
    new_order = non_pivot_cols + pivot_cols[:m]
    H_perm = H[:, new_order]
    
    # Apply RREF to permuted H
    H_rref, _ = gf2_rref(H_perm)
    
    # Extract P from systematic form [P | I_m]
    P = H_rref[:, :k]
    
    # Construct G = [I_k | P.T] in permuted order
    G_perm = np.hstack([np.eye(k, dtype=int), P.T])
    
    # Restore original column order
    G = np.zeros((k, n), dtype=int)
    for i, col_idx in enumerate(new_order):
        G[:, col_idx] = G_perm[:, i]
    
    return G.astype(np.int8)

# ============================================
# SETUP: H matrix and TWO G matrices
# ============================================

H = np.array([
    [1, 0, 1, 0, 1, 0],
    [0, 1, 1, 0, 0, 1],
    [1, 1, 0, 1, 0, 0]
], dtype=np.int8)

# WRONG G (from your original code)
G_wrong = np.array([
    [1, 0, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 0],
    [0, 0, 1, 0, 1, 1]
], dtype=np.int8)

print("Finding CORRECT G that satisfies H @ G.T = 0...")
G_correct = find_generator_from_H(H)
print("✓ Found valid G!\n")

print("="*70)
print("COMPARISON: WRONG G vs CORRECT G")
print("="*70)

# Verify both
print("\n1. VERIFICATION: H @ G.T")
print("-"*70)
result_wrong = (H @ G_wrong.T) % 2
result_correct = (H @ G_correct.T) % 2

print("WRONG G:")
print(G_wrong)
print("\nH @ G_wrong.T =")
print(result_wrong)
print(f"Status: {'❌ INVALID (not zero!)' if np.any(result_wrong != 0) else '✓ Valid'}")

print("\nCORRECT G:")
print(G_correct)
print("\nH @ G_correct.T =")
print(result_correct)
print(f"Status: {'✅ VALID (all zeros!)' if np.all(result_correct == 0) else '❌ Invalid'}")

# ============================================
# TEST 1: Encoding with NO NOISE
# ============================================

print("\n\n" + "="*70)
print("TEST 1: ENCODING WITHOUT NOISE")
print("="*70)

n = 6
k = 3
msg_bits = np.array([1, 0, 1], dtype=np.uint8)
print(f"\nOriginal message: {msg_bits}")

# Encode with WRONG G
encoded_wrong = (msg_bits @ G_wrong) % 2
print(f"\n--- Using WRONG G (H @ G.T ≠ 0) ---")
print(f"Encoded: {encoded_wrong}")

# Check syndrome (should be 0, but won't be!)
syndrome_wrong = (H @ encoded_wrong) % 2
print(f"Syndrome: {syndrome_wrong}")
print(f"Status: {'❌ NON-ZERO! Decoder sees this as CORRUPTED' if np.any(syndrome_wrong != 0) else '✓ Valid'}")

# Encode with CORRECT G
encoded_correct = (msg_bits @ G_correct) % 2
print(f"\n--- Using CORRECT G (H @ G.T = 0) ---")
print(f"Encoded: {encoded_correct}")

# Check syndrome
syndrome_correct = (H @ encoded_correct) % 2
print(f"Syndrome: {syndrome_correct}")
print(f"Status: {'✅ ALL ZEROS! Valid codeword' if np.all(syndrome_correct == 0) else '❌ Invalid'}")

# ============================================
# TEST 2: Decoding WITHOUT Noise
# ============================================

print("\n\n" + "="*70)
print("TEST 2: DECODING WITHOUT ANY NOISE")
print("="*70)
print("(Decoder should return original message perfectly)")

decoder = BpDecoder(H, error_rate=0.0, max_iter=100)

print("\n--- WRONG G ---")
try:
    decoded_wrong = decoder.decode(encoded_wrong)
    print(f"Original: {msg_bits}")
    print(f"Decoded:  {decoded_wrong[:k]}")
    match_wrong = np.array_equal(msg_bits, decoded_wrong[:k])
    print(f"Match: {'✅ YES (got lucky!)' if match_wrong else '❌ NO - Decoder tried to fix valid codeword!'}")
except Exception as e:
    print(f"❌ Decoding FAILED: {e}")
    match_wrong = False

print("\n--- CORRECT G ---")
try:
    decoded_correct = decoder.decode(encoded_correct)
    print(f"Original: {msg_bits}")
    print(f"Decoded:  {decoded_correct[:k]}")
    match_correct = np.array_equal(msg_bits, decoded_correct[:k])
    print(f"Match: {'✅ YES - Perfect!' if match_correct else '❌ NO'}")
except Exception as e:
    print(f"❌ Decoding FAILED: {e}")
    match_correct = False

# ============================================
# TEST 3: With REAL Noise - Performance Test
# ============================================

print("\n\n" + "="*70)
print("TEST 3: PERFORMANCE WITH NOISE (BER = 8%)")
print("="*70)

ber = 0.08
n_tests = 20
decoder = BpDecoder(H, error_rate=ber, max_iter=100)

print("\n--- Testing WRONG G ---")
wrong_g_failures = 0
for i in range(n_tests):
    msg = np.random.randint(0, 2, k, dtype=np.uint8)
    enc = (msg @ G_wrong) % 2
    noisy = add_noise(enc, ber=ber)
    try:
        dec = decoder.decode(noisy)
        if not np.array_equal(msg, dec[:k]):
            wrong_g_failures += 1
    except:
        wrong_g_failures += 1

print(f"Failures: {wrong_g_failures}/{n_tests} ({wrong_g_failures/n_tests*100:.0f}%)")

print("\n--- Testing CORRECT G ---")
correct_g_failures = 0
for i in range(n_tests):
    msg = np.random.randint(0, 2, k, dtype=np.uint8)
    enc = (msg @ G_correct) % 2
    noisy = add_noise(enc, ber=ber)
    try:
        dec = decoder.decode(noisy)
        if not np.array_equal(msg, dec[:k]):
            correct_g_failures += 1
    except:
        correct_g_failures += 1

print(f"Failures: {correct_g_failures}/{n_tests} ({correct_g_failures/n_tests*100:.0f}%)")

improvement = ((wrong_g_failures - correct_g_failures) / n_tests) * 100
if improvement > 0:
    print(f"\n📊 Correct G improved performance by {improvement:.0f}%!")
elif improvement < 0:
    print(f"\n⚠️ Unexpected: Wrong G performed better (very rare, statistical noise)")
else:
    print(f"\n➡️ Same performance (both might be working or failing)")

# ============================================
# EXPLANATION
# ============================================

print("\n\n" + "="*70)
print("WHY H @ G.T = 0 MATTERS")
print("="*70)

print("""
╔═══════════════════════════════════════════════════════════════════╗
║  WHEN H @ G.T ≠ 0 (WRONG G)                                      ║
╚═══════════════════════════════════════════════════════════════════╝

1. Encoder creates "invalid" codewords
   → Syndrome = H @ codeword ≠ 0 (even without noise!)
   
2. Decoder sees NON-ZERO syndrome
   → Thinks: "This codeword has errors!"
   → Tries to "fix" a perfectly valid codeword
   
3. Two sources of "errors":
   → Built-in violations from wrong G
   → Actual noise from channel
   → Decoder can't distinguish them!
   
4. Result: UNRELIABLE
   → Sometimes works by accident
   → Often fails even without noise
   → Poor error correction performance

╔═══════════════════════════════════════════════════════════════════╗
║  WHEN H @ G.T = 0 (CORRECT G)                                    ║
╚═══════════════════════════════════════════════════════════════════╝

1. Encoder creates VALID codewords
   → Syndrome = H @ codeword = 0 (without noise)
   
2. Decoder sees ZERO syndrome (no noise case)
   → Thinks: "Perfect! No errors."
   → Doesn't modify anything
   
3. Only one source of errors:
   → Actual noise from channel
   → Decoder can focus on fixing real errors
   
4. Result: RELIABLE
   → Always works without noise
   → Good error correction with noise
   → Predictable performance

╔═══════════════════════════════════════════════════════════════════╗
║  ANALOGY                                                          ║
╚═══════════════════════════════════════════════════════════════════╝

Wrong G = Selling a product with manufacturing defects
  → Quality control (decoder) sees defects as damage
  → Tries to "repair" brand new products
  → Can't tell factory defects from shipping damage
  → System doesn't work

Correct G = Selling perfect products
  → Quality control only flags actual damage
  → Repairs only what's actually broken
  → System works as designed
""")

print("="*70)
print("CONCLUSION: H @ G.T = 0 is MANDATORY!")
print("="*70)